# Introduction

So... you've got some interesting property of the network uncovered. Is it real? How can you trust that what you've found **didn't arise by random chance**?

One useful way of thinking by using generative models of random graphs. By "generative" and "random", we mean that the graph was generated using some statistical random model underlying it. This is essentially in the "Bayesian" tradition of modelling.

The alternative is a non-parametric way of thinking. This is where we don't assume some model that generated the data, and instead performs randomization on the data on hand to calculate statistics. Here, we assume that our randomization procedure is an appropriate random model for the network.

## Statistics Primer

**Discrete Distributions:**

- Bernoulli: probability p of success in 1 try (e.g. one flip of coin).
- Binomial: probability p of success given n number of tries. `n * bernoulli trials` follows binomial distribution.
- Poisson: processes with a per-unit rate.

**Continuous Distributions:**

- Uniform: equal probability over the range of probable values. Can also be made discrete.
- Normal: everyone's favourite.

We can also make up an arbitrary distribution of our own. :-)

## Statistical Test of Significance of Busy Routes

If we think back to the graph made in the previous notebook, what kind of process might one imagine that gave rise to the data?

**brainstorm session...**

Group work.

## Discrete uniform distribution over all routes

Assumptions of this model:

1. Riders choose start station with uniform probability.
2. They then choose a destination station with uniform probability.
3. They ride it and the data are recorded.

Under this model, what is the distribution of number of rides taken throughout the ? We can probably solve this analytically, but since we have ultra-cheap computational capabilities at our fingertips, might as well do it the brute-force way.

In [ ]:
import networkx as nx

G = nx.read_gpickle('datasets/divvy_2013/divvy_graph.pkl')
total_trips = sum([d['count'] for _,_,d in G.edges(data=True)])
print(total_trips)

### Exercise

Calculate the expected number of trips between any two stations, assuming self-loops are allowed.

In [ ]:
float(total_trips) / len(G.nodes()) ** 2

### Exercise

Can you plot a histogram of the number of trips taken on each route?

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

num_trips = [d['count'] for _,_,d in G.edges(data=True)]

plt.bar(Counter(num_trips).keys(), Counter(num_trips).values())

### Exercise

Can you figure out the 2.5th, 97.5th, and 100th percentile of this distribution?

Hint: You may wish to use `numpy`'s `percentile` function.

In [ ]:
import numpy as np

np.percentile(num_trips, [2.5, 97.5, 100])

Computing the interval between the 2.5th to the 97.5th percentile effectively gives you a centered 95% mass of the distribution. Knowing this range can be quite useful in other data analysis cases.

### Live Class Exercise

Together, we will implement a random model for the number of trips that are taken between two stations.

In this procedure, we will copy the nodes graph `G` into a new graph `G_random`. This can be done by:

    G_random = nx.Graph()
    G_random.add_nodes_from(G.nodes(data=True))
    
Following that, we will manually map-reduce this computation.

In [ ]:
# Grab all the nodes from G.
G_random = nx.Graph()
G_random.add_nodes_from(G.nodes(data=True))
G_random.nodes(data=True)

In [ ]:
# Implement the procedure by brute force. You will find it takes a bit of time.......... but because we have so many people 
# in the class, we can brute force parallelize this :-).
from random import sample, choice
import math

n = 40 # the total number of students with Python 3.4 installed.

for i in range(math.ceil(total_trips/n)):
    if i % 1000 == 0:
        print(i)
    n1 = choice(G_random.nodes())
    n2 = choice(G_random.nodes()) #note: n1 and n2 might well be the same nodes.
    if (n1, n2) not in G_random.edges():
        G_random.add_edge(n1, n2, count=0)
    else:
        G_random.edge[n1][n2]['count'] += 1

In [ ]:
your_id = 1 # change this number to the ID given to you.
nx.write_gpickle(G_random, 'datasets/divvy_2013/divvy_random{0}.pkl'.format(your_id))

In [ ]:
# Load the graphs with randomly re-distributed trip counts as a list of graphs.
import os

def is_pkl(filename):
    """
    Checks if a file is a pickled graph.
    """
    if filename.split('.')[-1] == 'pkl':
        return True
    else:
        return False
    
def is_divvy_random_graph(filename):
    """
    Checks if it's a Divvy graph.
    """
    if 'divvy_random' in filename:
        return True
    else:
        return False
    
# Load the data into a list of graphs.
divvy_random_graphs = []
for f in os.listdir('datasets/divvy_2013'):
    if is_pkl(f) and is_divvy_random_graph(f):
        g = nx.read_gpickle('datasets/divvy_2013/{0}'.format(f))
        divvy_random_graphs.append(g)
        
print(len(g.nodes()))

In [ ]:
# Plot the distribution of counts in the simulated re-distributed set of trips.
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from collections import Counter

G_random_master = nx.Graph()

matrix_master = np.zeros((300, 300)) # shape of matrix is n_nodes by n_nodes
for g in divvy_random_graphs:
    # Inspired by Jake VDP's talk on using numpy operations to speed up code...
    # Recall that graphs can be expressed as a matrix. 
    m = nx.to_numpy_matrix(g, weight='count') # can specify what number will be used in each numpy cell.
    # Sum up the matrices, in a vectorized fashion.
    matrix_master += m
    
    # Note: This is much faster compared to iterating over all the edges. If you deal with large graphs,
    # then practically speaking, it may be better to work with matrices rather than graph objects.
    
# Reshape the matrix to make it easy for plt.hist to handle it.
data = matrix_master.reshape(300**2)
# plt.hist(data) # OR
plt.bar(Counter(data).keys(), Counter(data).values())

Here is an attempt at writing a numpy-vectorized version of the simulation.

Once again, this is inspired by Jake VDP's PyCon talk on using numpy.

The core idea here is as such:

1. Assume all possible edges had a unique index.
2. We can store this in an array of size n_nodes^2.
3. We can generate random integers in a second array of size `total trips`, but with maximum value `edges`.
4. We then create a histogram using `np.histogram(trip_edges, bins=num_edges+1)`. This returns two arrays:
    1. an array with the number of trips in each edge
    2. an array with the bin numbers (representing the edges).

This here will be much faster than brute-force iteration because we use `numpy` functions.

Special thanks to Mauris Van Hauwe on Stack Overflow: http://stackoverflow.com/questions/29578781/how-to-vectorize-simulation-of-distributing-coins-in-numpy/29578883#29578883

In [ ]:
# Total number of possible edges.
edges = len(G.nodes())**2

# Recall that the total trips was recorded in the variable total_trips
total_trips

# coins = 800000
# slots = 900000

trip_edges = np.random.randint(edges, size=total_trips) 
trips_in_each_edge = np.histogram(trip_edges, bins=np.arange(edges+1))[0]
trips_in_each_edge
plt.hist(trips_in_each_edge)
print(np.mean(trips_in_each_edge), np.var(trips_in_each_edge))

**Think about it...**

Given the distribution of trip counts in the randomly re-distributed trips, what can we infer about the popularity of certain routes? Are they likely to have shown up in this random model?